In [45]:
import pandas as pd
import numpy as np
import re
import os
import glob
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model

In [41]:
#os dados de cada Eleição (separados por turno) estão no df dataElecs
pathElec2012_1t = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\1Turno2012'
pathElec2012_2t = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\2Turno2012'
pathElec2014_1t_gov = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\1Turno14-Proc\\Governador'
pathElec2014_1t_pres = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\1Turno14-Proc\\Presidente'
pathElec2014_2t_gov = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\2Turno14-Proc\\Governador'
pathElec2014_2t_pres = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\2Turno14-Proc\\Presidente'
pathElec2016_1t = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\1Turno2016'
pathElec2016_2t = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\2Turno2016'
pathElec2018_1t_gov = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\1Turno2018\\Governador'
pathElec2018_1t_pres = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\1Turno2018\\Presidente'
pathElec2018_2t_gov = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\2Turno2018\\Governador'
pathElec2018_2t_pres = 'C:\\Users\\thiag\\Desktop\\Thiago\\UnB\\Fundamentos de Sistemas Inteligentes\\ProjetoFinal\\2Turno2018\\Presidente'

paths = [pathElec2012_1t, pathElec2012_2t, pathElec2014_1t_gov, pathElec2014_1t_pres,
         pathElec2014_2t_gov, pathElec2014_2t_pres, pathElec2016_1t,pathElec2016_2t, pathElec2018_1t_gov, pathElec2018_1t_pres,
         pathElec2018_2t_gov, pathElec2018_2t_pres]

dataElecs_AC = []
dataElecs_DC = []
j=0
for path in paths:
    os.chdir(path)
    if (j<8):
        filenames = glob.glob("bweb_*.txt")
        dfs = [pd.read_table(filename,encoding = 'ISO-8859-1', delimiter = ',') for filename in filenames]
    else:
        filenames = glob.glob("bweb_*.csv")
        dfs = [pd.read_csv(filename,encoding = 'ISO-8859-1', delimiter = ',') for filename in filenames]
    i=0
    df_certo = []
    for df in dfs:
        UF_name = filenames[i]
        UF_name = UF_name[8:10]
        d = {'Sigla da Unidade da Federação':[UF_name]*(df.shape[0])}
        UF = pd.DataFrame(data = d)
        df_certo.append(pd.concat([df,UF], ignore_index=False, sort= False,axis=1))
        i = i+1
    data = pd.concat(df_certo, ignore_index=True,sort= False)
    #import pdb; pdb.set_trace()
    data.columns.values[0] = 'Nome_Munic'
    data['Nome_Munic'] = data['Nome_Munic'].astype('str')
    data['Nome_Munic'] = data['Nome_Munic'].str.upper()
   # import pdb; pdb.set_trace()
    data['Nome_Munic'] = data['Nome_Munic'].map(lambda x: re.sub(r'[^\w\s]', '', x))
    data['Nome_Munic'] = data['Nome_Munic'].str.replace(' ', '')
    nomeMunicEAbstencoes = pd.concat([data['Nome_Munic'],data['Sigla da Unidade da Federação'],data['Resultados_5']], ignore_index = True, axis = 1)
    nomeMunicEAbstencoes.columns = ['Nome_Munic','Sigla da Unidade da Federação', 'Abstencoes']
    dataElecs_AC.append(data)
    dataElecs_DC.append(nomeMunicEAbstencoes)
    j = j+1
    
a = dataElecs_DC[0]
for i in range(0,len(dataElecs_DC)-1):    
    a = pd.merge(a,dataElecs_DC[i+1], on = ['Nome_Munic', 'Sigla da Unidade da Federação'],how='left')
    i = i+1
    
a.mean(axis = 1)

absMediaPorMunic = pd.concat([a['Nome_Munic'], a['Sigla da Unidade da Federação'] , a.mean(axis=1)],axis=1)
absMediaPorMunic.columns.values[2] = 'AbsMedia'

In [42]:
#1° turno 2016
dataElec2016_1t = pd.concat([dataElecs_AC[6]['Nome_Munic'],dataElecs_AC[6]['Sigla da Unidade da Federação'],dataElecs_AC[6]['Resultados_5'] ,dataElecs_AC[6]['Resultados_2'], dataElecs_AC[6]['Resultados_4'], dataElecs_AC[6]['Resultados_8'], dataElecs_AC[6]['Resultados_9']],axis=1)
y_aux = pd.merge(dataElec2016_1t, absMediaPorMunic, on= ['Nome_Munic' , 'Sigla da Unidade da Federação'])
y = y_aux['Resultados_5'].sub(y_aux['AbsMedia'], fill_value = 0)
X = y_aux.iloc[:,3:7]
for i in range(0,X.shape[1]):
    if (X.iloc[:,i].dtype == 'O'):
        X.iloc[:,i] = pd.to_numeric(X.iloc[:,i], errors = 'coerce')

X = X.replace('\.+', np.nan, regex=True)
X = X.replace(',','.',regex = True)
X.fillna(0,inplace = True)        
X = StandardScaler().fit_transform(X)

In [46]:
regressors = [
    RandomForestRegressor(n_estimators = 500, max_depth = 100),
    KNeighborsRegressor(n_neighbors=15),
    linear_model.Lars(),
    MLPRegressor(hidden_layer_sizes = (50,), activation = 'logistic', alpha = 0.01),
    SVR(kernel = 'linear', C=1),
    SVR(kernel = 'rbf', C=1), 
]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

scores = []


for reg in regressors:
    reg.fit(X_train,y_train)
    scores.append(reg.score(X_test,y_test))
scores

[-0.04132904664389092,
 0.025074038763352346,
 0.03904885274152492,
 0.003730328196893895,
 -0.00015091910453768342,
 -0.00015091910453768342]

In [39]:
#1° turno 2014
dataElec2014_1t = pd.concat([dataElecs_AC[0]['Nome_Munic'],dataElecs_AC[0]['Sigla da Unidade da Federação'],dataElecs_AC[0]['Resultados_5'] ,dataElecs_AC[0]['Resultados_2'], dataElecs_AC[0]['Resultados_4'], dataElecs_AC[0]['Resultados_8'], dataElecs_AC[0]['Resultados_9']],axis=1)
y_aux = pd.merge(dataElec2012_1t, absMediaPorMunic, on= ['Nome_Munic' , 'Sigla da Unidade da Federação'])
y = y_aux['Resultados_5'].sub(y_aux['AbsMedia'], fill_value = 0)
X = y_aux.iloc[:,3:7]
for i in range(0,X.shape[1]):
    if (X.iloc[:,i].dtype == 'O'):
        X.iloc[:,i] = pd.to_numeric(X.iloc[:,i], errors = 'coerce')

X = X.replace('\.+', np.nan, regex=True)
X = X.replace(',','.',regex = True)
X.fillna(0,inplace = True)        
X = StandardScaler().fit_transform(X)

In [ ]:
regressors = [
    #RandomForestRegressor(n_estimators = 100, max_depth = 100),
    RandomForestRegressor(n_estimators = 50, max_depth = 10),
    KNeighborsRegressor(n_neighbors=15)
    
]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

scores = []


for reg in regressors:
    reg.fit(X_train,y_train)
    scores.append(reg.score(X_test,y_test))
scores